This script is used to 1.) compare the omics differential network, 2.) observe the common pathways between the omics differential networks, 3.) examine how the integration affects whether an edge becomes significant or not compared to the single omics only.

### Comparing with the significant edges BETWEEN the differential networks for all omics

In [ ]:
def edge_num (omics):
    with open('../Data/permutation_test_files_'+omics+'/sig_edges.txt') as f:
        lines = f.readlines()
    edges_remaining = []
    edges = lines[0].split(",")

    for index in range(0,len(edges),2):
        list1 = edges[index],(edges[index+1][1:]) #becomes a tuple
        edges_remaining.append(list1)

    return edges_remaining


In [ ]:
proteomic_edges = edge_num('proteomics')
integrated_edges = edge_num('integrated')
metabolomic_edges = edge_num('metabolomics')

In [ ]:
intersection = list(set(metabolomic_edges).intersection(list(set(proteomic_edges))))  
len(intersection) 

In [ ]:
intersection = list(set(integrated_edges).intersection(list(set(metabolomic_edges))))  
len(intersection) 

In [ ]:
intersection = list(set(integrated_edges).intersection(list(set(proteomic_edges))))  
len(intersection) 

In [ ]:
intersection = (set(metabolomic_edges) & set(proteomic_edges) & set(integrated_edges))  
len(intersection) 

In [ ]:
#Pathway pairs for integrated data not detected by either single omics
len(list(set(integrated_edges) - (set(metabolomic_edges)|set(proteomic_edges)))) 

### Get number of common pathways between the three omics

In [ ]:
import pandas as pd
import sspa

In [ ]:
metabolomic_df = pd.read_csv('../Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
proteomic_df = pd.read_csv('../Data/Su_COVID_proteomics_processed_commoncases.csv', index_col=0)
integrated_df = pd.read_csv("../Data/Su_integrated_data.csv", index_col=0)

#Remove root pathways
#Convert pathway ID to name
root_path = pd.read_excel('../Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())

Load in all the Reactome pathway definitions for all three omics:

In [ ]:
reactome_pathways = sspa.process_gmt("../Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")
kpca_scores = sspa.sspa_kpca(metabolomic_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

metabolomic_pathways = kpca_scores.columns
metabolomic_pathways

In [ ]:
#Load in a file downloaded from https://reactome.org/download/current/UniProt2Reactome_All_Levels.txt
reactome_pathways = sspa.process_reactome('Homo sapiens', infile = '../Data/UniProt2Reactome_All_Levels_ver84.txt', download_latest = False, filepath = None)
kpca_scores = sspa.sspa_kpca(proteomic_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

proteomic_pathways = kpca_scores.columns
proteomic_pathways

In [ ]:
reactome_pathways = pd.read_csv("../Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")
kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

integrated_pathways = kpca_scores.columns
integrated_pathways

In [ ]:
print(len(set(metabolomic_pathways).intersection(list(set(proteomic_pathways)))))   #47 pathways
print(len(set(metabolomic_pathways) & set(proteomic_pathways) & set(integrated_pathways)))  #47 pathways 

print(len(list(set(metabolomic_pathways).intersection(list(set(integrated_pathways))))))  #144 pathways (same as number of metabolomic)
print(len(list(set(proteomic_pathways).intersection(list(set(integrated_pathways)))))) #578 pathways (same as number of proteomic)

#710 pathways in integrated dataset
#((144+578)-47+35)

len(list(set(integrated_pathways) - (set(metabolomic_pathways)|set(proteomic_pathways)))) #35 pathways for integrated data not detected by either single omics

Obtain new pathways made after the integration of the Reactome metabolomic and proteomic pathway definitions i.e. pathways comprised of one metabolite and one protein.

In [ ]:
new_pathways = list(set(integrated_pathways) - (set(metabolomic_pathways)|set(proteomic_pathways)))
new_pathways

In [ ]:
#Make a dictionary of the molecules present in the integrated Reactome definitions and in the integrated dataset

df = pd.read_csv("../Data/Su_integrated_data.csv", index_col=0)
reactome_pathways = pd.read_csv("../Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")


kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

#Obtain pathways and corresponding metabolites for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}

#Filter out the compounds in the pathways that are not present in the dataset

#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

In [ ]:
#All 35 new pathways comprised of one metabolite and one protein

proteomic_only_pathway = []
proteomic_pathway_with_onemetabolite = []
metabolomic_only_pathway = []
metabolomic_pathway_with_oneprotein = []
both_omic = []
new_pathway = []

for key,values in filtered_dict.items():
    #if key in new_pathways:
        #print(values)
    #print(values)
    proteomic=False
    metabolomic=False
    both=False


    proteins = 0
    metabolites = 0

    for value in values:
     
        if value[0].isalpha():
            proteins += 1
        else:
            metabolites += 1

    
    if metabolites == 0:
        proteomic_only_pathway.append(key)
    if proteins > 1 and metabolites == 1:
        proteomic_pathway_with_onemetabolite.append(key)
    
    if proteins == 0:
        metabolomic_only_pathway.append(key)
    if metabolites > 1 and proteins == 1:
        metabolomic_pathway_with_oneprotein.append(key)

    if metabolites > 1 and proteins > 1:
        both_omic.append(key)
    if proteins == 1 and metabolites == 1:
        new_pathway.append(key)



In [ ]:
print(len(proteomic_only_pathway))
print(len(proteomic_pathway_with_onemetabolite))
print(len(metabolomic_only_pathway))
print(len(metabolomic_pathway_with_oneprotein))
print(len(both_omic))
print(len(new_pathway))

### Comparing the initial test statistics of the three omics to see which edges were affected

In [ ]:
import pandas as pd

Comparing the initial test statistics of the integrated data with the proteomic initial test statistics:

In [ ]:
metabolomic = pd.read_csv('../Data/permutation_test_files_metabolomics/observed_tstats.csv', index_col=0)
proteomic = pd.read_csv('../Data/permutation_test_files_proteomics/observed_tstats.csv', index_col=0)
integrated = pd.read_csv('../Data/permutation_test_files_integrated/observed_tstats.csv', index_col=0)

In [ ]:
metabolomic = metabolomic.rename(columns={'Observed_tstat': 'metabolomic_tstat'})
proteomic = proteomic.rename(columns={'Observed_tstat': 'proteomic_tstat'})
integrated = integrated.rename(columns={'Observed_tstat': 'integrated_tstat'})

Comparing the initial test statistics of the integrated data with the proteomic initial test statistics:

In [ ]:
initial_tstat_comp = proteomic.join(integrated) 
initial_tstat_comp #length of proteomic_tstat

In [ ]:
#Number of pathway pairs with initial test statistic that are the same

proteomic_only_edge = []
metabolomic_influence_edge = []
counter = 0

for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].proteomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        proteomic_only_edge.append(edge)
        counter += 1
    else:
        metabolomic_influence_edge.append(edge)


print(counter)

#166,753 edges of the proteomic initial test statistic dataset are all included in the integrated initial test statistic data
#117,381 edges out of 166,753 are identical
#Therefore 49,372 edges were influenced by the metabolomic data

len(metabolomic_influence_edge)

In [ ]:
#Change to tuple format so you can compare with the significant edges in the differential networks

metabolomic_influence_edge_tuple = []
for i in range(len(metabolomic_influence_edge)):
    edge = metabolomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    metabolomic_influence_edge_tuple.append(edge)


proteomic_only_edge_tuple  = []
for i in range(len(proteomic_only_edge )):
    edge = proteomic_only_edge [i].split(",")
    edge = (edge[0],edge[1][1:])
    proteomic_only_edge_tuple.append(edge)

In [ ]:
#Need to see proportion of metabolomic-influenced edges that are significantly differentially expressed edges (from differential network) in the proteomic data vs the integrated data
print(len(metabolomic_influence_edge))
print(len(list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple))))) #Before integrating with the metabolomics data, 680 edges significant
print(len(list(set(integrated_edges).intersection(set(metabolomic_influence_edge_tuple))))) #After integrating with the metabolomics data, 765 significant


#290 edges are the same between 680 edges and 765 edges
list1 = list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple)))
list2 = list(set(integrated_edges).intersection(set(metabolomic_influence_edge_tuple)))
len(list(set(list1).intersection(set(list2))))

In [ ]:
#Showing that 'Metabolomic influenced' proteomic pathways that are significant in the differential network are instances where at least one pathway is a common pathway OR one pathway is influenced by one metabolite (not enough to be considered a pathway in the metabolomic set)

#proteomic_edges = edges significant in proteomic differential network
list1 = list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple)))

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
print(len(common_pathways))
counter = 0 

influenced_noncommon_pathway_pair = []
for edges in list1:
    pathway_pair = list(edges)
    if (pathway_pair[0] in common_pathways) or (pathway_pair[1] in common_pathways):
        #print(pathway_pair)
        counter += 1
    else:
        influenced_noncommon_pathway_pair.append(edges)


print(counter)
#327 pathway pairs have at least one common pathway in them, out of the differentially expressed protein pathways in the network that are influenced by the metabolomics dataset
#The other cases are where there is one metabolite added to one of the pathways that is not enough to be constituted as a pathway in the metabolomics dataset

In [ ]:
#We can look at the pathways before and after integration to prove that for the non-common pathways that are influenced by the other omics, there is only one analyte from the other dataset added to it

df = pd.read_csv("../Data/Su_integrated_data.csv", index_col=0)
reactome_pathways = pd.read_csv("../Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")


kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

#Obtain pathways and corresponding metabolites for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}

#Filter out the compounds in the pathways that are not present in the dataset

#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

In [ ]:
#As an example, look at pathway 'R-HSA-422475'

print(influenced_noncommon_pathway_pair)
list(reactome_pathways.loc['R-HSA-422475'])  #All proteins, only one metabolite
filtered_dict['R-HSA-422475']

Comparing the initial test statistics of the integrated data with the metabolomic initial test statistics:

In [ ]:
initial_tstat_comp = metabolomic.join(integrated)
initial_tstat_comp #length of metabolomic_tstat

In [ ]:
#Number of pathway pairs with initial test statistic that are the same
metabolomic_only_edge = []
proteomic_influence_edge = []

counter = 0
for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].metabolomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        metabolomic_only_edge.append(edge)
        counter += 1
    else:
        proteomic_influence_edge.append(edge)

print(counter)

#10,296 edges of the metabolomic initial test statistic dataset are all included in the integrated initial test statistic data
#2,088 edges out of 10,296 are identical
#Therefore 8,208 edges were influenced by the proteomic data

len(proteomic_influence_edge)

In [ ]:
#Change to tuple format so you can compare with the significant edges in the differential networks

proteomic_influence_edge_tuple = []
for i in range(len(proteomic_influence_edge)):
    edge = proteomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    proteomic_influence_edge_tuple.append(edge)


metabolomic_only_edge_tuple = []
for i in range(len(metabolomic_only_edge)):
    edge = metabolomic_only_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    metabolomic_only_edge_tuple.append(edge)

In [ ]:
#Need to see proportion of proteomic-influenced edges that are significant edges in the metabolomic data vs the integrated data
print(len(proteomic_influence_edge))
print(len(list(set(metabolomic_edges).intersection(set(proteomic_influence_edge_tuple))))) #Before integrating with the proteomics data, 17 edges significant
print(len(list(set(integrated_edges).intersection(set(proteomic_influence_edge_tuple))))) #After integrating with the proteomics data, 45 significant

#2 edges are the same between 17 edges and 45 edges
list1 = list(set(metabolomic_edges).intersection(set(proteomic_influence_edge_tuple)))
list2 = list(set(integrated_edges).intersection(set(proteomic_influence_edge_tuple)))
len(list(set(list1).intersection(set(list2))))

### Testing the integrated only edges

In [ ]:
integrated_only = list(set(integrated_edges) - (set(metabolomic_edges)|set(proteomic_edges)))
len(integrated_only)

In [ ]:
print(len(proteomic_only_pathway))
print(len(proteomic_pathway_with_onemetabolite))
print(len(metabolomic_only_pathway))
print(len(metabolomic_pathway_with_oneprotein))
print(len(both_omic))
print(len(new_pathway))

In [ ]:
#Pathways where one is a common pathway

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in both_omic) or (pathway_pair[1] in both_omic):
        #print(pathway_pair)
        counter += 1

print(counter)

In [ ]:
#Pathways where one is a common pathway and one is a new pathway
 
counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in both_omic) and (pathway_pair[1] in new_pathway):
        counter += 1
    if (pathway_pair[1] in both_omic) and (pathway_pair[0] in new_pathway):
        counter += 1

print(counter)

In [ ]:
#Pathways where one is a new pathway

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in new_pathway) or (pathway_pair[1] in new_pathway):
        counter += 1

print(counter)

In [ ]:
#Pathways where both are the same

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    #if (pathway_pair[0] in metabolomic_only_pathway) and (pathway_pair[1] in metabolomic_only_pathway):
    #   counter += 1
    if (pathway_pair[0] in proteomic_only_pathway) and (pathway_pair[1] in proteomic_only_pathway):
        counter += 1
    #if (pathway_pair[0] in metabolomic_pathway_with_oneprotein) and (pathway_pair[1] in metabolomic_pathway_with_oneprotein):
    #    counter += 1
    #if (pathway_pair[0] in proteomic_pathway_with_onemetabolite) and (pathway_pair[1] in proteomic_pathway_with_onemetabolite):
    #    counter += 1

print(counter)

In [ ]:
counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in proteomic_pathway_with_onemetabolite) and (pathway_pair[1] in proteomic_only_pathway):
        counter += 1
    if (pathway_pair[1] in proteomic_pathway_with_onemetabolite) and (pathway_pair[0] in proteomic_only_pathway):
        counter += 1

print(counter)

In [ ]:

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in metabolomic_pathway_with_oneprotein) and (pathway_pair[1] in metabolomic_only_pathway):
        counter += 1
    if (pathway_pair[1] in metabolomic_pathway_with_oneprotein) and (pathway_pair[0] in metabolomic_only_pathway):
        counter += 1

print(counter)

In [ ]:

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in metabolomic_pathway_with_oneprotein) and (pathway_pair[1] in proteomic_only_pathway):
        counter += 1
    if (pathway_pair[1] in metabolomic_pathway_with_oneprotein) and (pathway_pair[0] in proteomic_only_pathway):
        counter += 1

print(counter)

In [ ]:

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in proteomic_pathway_with_onemetabolite) and (pathway_pair[1] in metabolomic_only_pathway):
        counter += 1
    if (pathway_pair[1] in proteomic_pathway_with_onemetabolite) and (pathway_pair[0] in metabolomic_only_pathway):
        counter += 1

print(counter)

In [ ]:
counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in proteomic_pathway_with_onemetabolite) and (pathway_pair[1] in metabolomic_pathway_with_oneprotein):
        counter += 1
    if (pathway_pair[1] in proteomic_pathway_with_onemetabolite) and (pathway_pair[0] in metabolomic_pathway_with_oneprotein):
        counter += 1

print(counter)

In [ ]:


counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in proteomic_only_pathway) and (pathway_pair[1] in metabolomic_only_pathway):
        #print(pathway_pair)
        counter += 1
    if (pathway_pair[0] in metabolomic_only_pathway) and (pathway_pair[1] in proteomic_only_pathway):
        #print(pathway_pair)
        counter += 1

print(counter)

### Test the spurious findings

In [ ]:
import os 
import numpy as np
import statsmodels.stats.multitest
from itertools import compress

In [ ]:
#Unusual findings: 
 
#Proteomic influenced only (i.e. pairs where both pathways are only in the proteomic dataset)
print(len(list(set(proteomic_only_edge_tuple).intersection(set(integrated_only)))))

#Metabolomic influenced only (i.e. pairs where both pathways are only in the metabolomic dataset)
print(len(list(set(metabolomic_only_edge_tuple).intersection(set(integrated_only)))))

In [ ]:
#Check the significance of these spurious results (don't use the term false positives)

proteomic_spurious = list(set(proteomic_only_edge_tuple).intersection(set(integrated_only)))
metabolomic_spurious = list(set(metabolomic_only_edge_tuple).intersection(set(integrated_only))) #0 edges

In [ ]:
path = os.getcwd()[:-13] + '\\Data\\permutation_test_files_proteomics\\Values'

val_array = []


for filename in os.listdir(path): #also lists directories
    with open(os.path.join(path, filename)) as file:    
        lines = file.readlines()
        vals = lines[0].split(';')
        vals =  [int(x) for x in vals]
        #print(vals)
        if filename == 'vals1.txt':
            val_array = np.array(vals)
        else:
            val_array = np.vstack([val_array, vals])

val_array
val_array2 = val_array.sum(axis=0) #add up the values by columns for each pathway pair
pval_array = val_array2 / 100000


In [ ]:
sig_edge_boolean,adjusted_pval = statsmodels.stats.multitest.fdrcorrection(pval_array, alpha=0, method='poscorr', is_sorted=False)

In [ ]:
df = pd.read_csv('../Data/permutation_test_files_proteomics/observed_tstats.csv', index_col=0)
edgelist = df.index

#sig_edges = list(compress(edgelist,sig_edge_boolean))


#Zip p-values to edge names to form dictionary (so I can test significance of differentially expressed genes)
edgelist_tuple = []
for i in range(len(edgelist)):
    edge = edgelist[i].split(",")
    edge = (edge[0],edge[1][1:])
    edgelist_tuple.append(edge)
    
#p_val_dict  = dict(zip(edgelist_tuple, adjusted_pval))
p_val_dict  = dict(zip(edgelist_tuple, pval_array))


In [ ]:
proteomic_spurious_pval = []
metabolomic_spurious_pval = []

for edge in proteomic_spurious:
    if edge in list(p_val_dict.keys()):
        proteomic_spurious_pval.append(p_val_dict[edge])

proteomic_spurious_pval[:100]

#Get the average p-value of the spurious findings
sum(proteomic_spurious_pval)/len(proteomic_spurious_pval)  
#Close to the 1e-5 p-value as expected